# Финальный проект - Космические братья

## Введение

**Описание проекта**

Мы занимаемся аналитикой для проекта Космические братья. Это новая игра, в которой пользователи могут строить разные здания и сражаться с другими пользователями.

Отдел маркетинга попросил нас разобраться в источниках, которые мы используем для привлечения пользователей. В долгосрочном периоде мы хотим оптимизировать расходы на рекламу, поэтому нам необходимо определить какие источники работают хорошо, а какие источники - плохо.

Пока что в игре еще не реализована монетизация, но мы планируем зарабатывать на пользователях, показывая рекламу во время стороительства зданий. Для определения эффективных каналов мы можем использовать промежуточные метрики, например: время в игре, количество построенных зданий, факт завершения первого уровня, и пр.

**Описание данных**

Датасет `game_actions.csv` - в нем представлены данные по игре пользователей на первом уровне. Завершение первого уровня требует от игрока выполнения одного из двух условий: 1) победа над первым врагом (PvP) 2) реализация проекта - разработка орбитальной сборки спутников (PvE).

В датасете содержатся данные первых пользователей приложения — когорты пользователей, которые начали пользоваться приложением в период с 4 по 10 мая включительно. Можем предположить, что день, когда пользователь открыл приложение - это день, когда он увидел рекламу.

1. `event_datetime` — время события
2. `event` — одно из трёх событий
3. `building_type` — один из трёх типов здания
4. `user_id` — идентификатор пользователя
5. `project_type` — тип реализованного проекта

Помимо основного датасета есть два датасета с информацией о рекламных активностях. Они также помогут в решении задачи.

Датасет `ad_costs.csv` - в нем представлены затраты на привлечения пользователей по дням и источникам. Чтобы привлечь пользователей 4 мая, реклама была оплачена 3 мая и т.д.

1. `day` - день, в который был совершен клик по объявлению
2. `source` - источник трафика
3. `cost` - стоимость кликов

Датасет `user_source.csv` содержит колонки:

1. `user_id` - идентификатор пользователя
2. `source` - источников, с которого пришёл пользователь, установивший приложение

**План работы**

1. Загрузить и подготовить данные
   1. Почистить колонки, задать типы данных
   2. Обработать пропуски и дубликаты
   3. Найти и исправить ошибки (если есть) 
2. Провести исследовательский анализ и рассчитать
   1. Количество (и доли) пользователей за весь период, тех кто прошел игру, по источникам трафика
   2. Количество (и доли на пользователя) событий за весь период, реализованных проектов, построенных зданий
   3. Количество (и доли) использованных источников трафика, стоимость клика по источникам
   4. Время прохождения уровня
3. Рассчитать метрики
   1. DAU, WAU и Sticky Factor (SF)
   2. LTV (с предположениями), CAC
4. Проверить гипотезы
   1. Время завершения уровня различается в зависимости от способа прохождения
   2. Количество построек зависит от канала привлечения
5. Сделать выводы и рекомендации


In [33]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats as st
from IPython.display import display

# Save raw datasets in case we need them
try:
    raw_actions = pd.read_csv('game_actions.csv')
    raw_costs = pd.read_csv('ad_costs.csv')
    raw_sources = pd.read_csv('user_source.csv')
except:
    raw_actions = pd.read_csv('/datasets/game_actions.csv')
    raw_costs = pd.read_csv('/datasets/ad_costs.csv')
    raw_sources = pd.read_csv('/datasets/user_source.csv')

# Constants, which we will need later
FIG_WIDTH = 10
FIG_HEIGHT = 5


## Предобработка данных

Для начала проверим какие данные нам достались. Нас интересуют:

1. Названия колонок.
2. Неподходящие под тип данных значения в колонках.
3. Артефакты в данных и повторяющиеся строки.
4. Пропущенные значения.
5. Дополнительные колонки и типы данных.

Пойдем по порядку.

In [34]:
# Check columns
for dataset in [raw_actions, raw_costs, raw_sources]:
    print('-' * 50)
    dataset.info()


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135640 entries, 0 to 135639
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   event_datetime  135640 non-null  object
 1   event           135640 non-null  object
 2   building_type   127957 non-null  object
 3   user_id         135640 non-null  object
 4   project_type    1866 non-null    object
dtypes: object(5)
memory usage: 5.2+ MB
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   source  28 non-null     object 
 1   day     28 non-null     object 
 2   cost    28 non-null     float64
dtypes: float64(1), object(2)
memory usage: 800.0+ bytes
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>


Название колонок уже заданы нормально. У нас есть много пропущенных занчений в датасете `raw_actions`, но это может быть связано с тем, как были собраны данные. Мы на это посмотрим в следующих секциях. Наконец, надо будет проверить типы данных.

In [48]:
for column in ['event', 'building_type', 'project_type']:
    print(
        '-' * 50, '\n',
        raw_actions[column].value_counts(),
        sep=''
    )


--------------------------------------------------
building            127957
finished_stage_1      5817
project               1866
Name: event, dtype: int64
--------------------------------------------------
spaceport          59325
assembly_shop      54494
research_center    14138
Name: building_type, dtype: int64
--------------------------------------------------
satellite_orbital_assembly    1866
Name: project_type, dtype: int64


In [56]:
print(
    '-' * 50, '\n',
    raw_costs.source.value_counts(), '\n',
    '-' * 50, '\n',
    raw_sources.source.value_counts(),
    sep=''
)


--------------------------------------------------
facebook_ads               7
instagram_new_adverts      7
yandex_direct              7
youtube_channel_reklama    7
Name: source, dtype: int64
--------------------------------------------------
yandex_direct              4817
instagram_new_adverts      3347
facebook_ads               2726
youtube_channel_reklama    2686
Name: source, dtype: int64


У нас нету странных значений в записях: каналы в затратах совпадают с каналами в источниках, а события в `row_actions` адекватны.

Следующим шагом, проверим дубликаты.

In [67]:
for dataset in [raw_actions, raw_sources]:
    display(dataset[dataset.duplicated()])

,event_datetime,event,building_type,user_id,project_type
74891,2020-05-10 18:41:56,building,research_center,c9af55d2-b0ae-4bb4-b3d5-f32aa9ac03af,NaN


,user_id,source


У нас есть одинокая строка с полным дубликатом в `raw_actions`. Убьем ее: явно один и тот же пользватель не мог совершить одно и то же действия в одно и то же время.

Причешем наши датасеты.

In [71]:
# No changes to the raw datasets
df_costs = raw_costs.copy()
df_sources = raw_sources.copy()

# Clean raw_actions 
df_actions = (
    raw_actions
    .copy()
    .drop_duplicates()
    .astype({
        'event_datetime': 'datetime64[s]'
    })
)


## Исследовательский анализ данных